# 앙상블 (Ensemble) : Boosting

- 다양한 모델을 결합하여 예측 성능을 향상시키는 방법
    - 깊이가 얕은 결정트리를 사용해 이전 트리의 오차를 보정하는 방식
    - 순차적으로 경사하강법을 사용해 이전 트리의 오차를 줄여나감
        - 분류모델에서는 손실함수 Logloss를 사용해 오차를 줄임
        - 회귀모델에서는 손실함수 MSE를 사용해 오차를 줄임
    - Boosting 계열은 일반적으로 결정트리 개수를 늘려도 과적합에 강함
    - 대표적인 알고리즘(모델): GradientBoosting, HistGradientBoosting, XGBoost(DMLC), LightGBM(MS), CatBoost

### GradientBoosting

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
from sklearn.tree import DecisionTreeRegressor    # 결정트리 사용

In [14]:
class SimpleGradientBoostingClassifier:
    
    # 초기 파라미터 설정 값들을 전달할 예정. → 객체 생성시 초기화 함수 호출, 인스턴스 속성 설정
    def __init__(self, n_estimators=100, learning_rate=0.2, max_depth=3):    # 초기화 함수, 생성자 : 객체 생성시 초기화 함수 호출
        self.n_estimators = n_estimators      # 트리 개수, 모델 개수
        self.learning_rate = learning_rate    # 학습률, 학습 속도
        self.max_depth = max_depth            # 트리 깊이
        self.initial_log_odds = 0             # 초기 로그 오즈 = 초기 예측값값
        self.trees = []                       # 트리 리스트 = estimator 모음 배열
    
    def log_odds(self, p):
        # 확률값 → 로짓 변환 : 0-1 사이의 값을 펼쳐 -무한대~+무한대 사이의 값으로 보정정
        # 확률값을 로그로 변환해줄 예정. 0-1 사이의 확률값을 로그로 변환해줄 예정.
        return np.log(p / (1 - p))

    def sigmoid(self, z):     # 시그모이드 함수 : 0~1 사이의 값을 반환, z값을 받아서 확률값으로 변환 → 이진분류를 위해서 만들어준다.
        # 로짓 변환값 → 확률값 변환 : 로짓 변환값을 시그모이드 함수에 넣어서 확률값으로 변환, 0-1 사이의 값으로 변환
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # 초기 예측값 설정
        y_mean = np.mean(y) # 초기 예측 확률값 평균으로 설정 → 이진 분류를 해결할 수 있도록 해준다.
        self.initial_log_odds = self.log_odds(y_mean) # 확률값은 0-1 사이의 작은 값이기 때문에 큰 범위의 로그 오즈로 변환해준다. -무한대 ~ +무한대
        y_pred_log_odds = np.full_like(y, self.initial_log_odds, dtype=np.float64) # 초기 예측값 배열 생성

        for _ in range(self.n_estimators):
            # 현재 상태에서 예측된 확률값 계산
            y_pred_proba = self.sigmoid(y_pred_log_odds)
            
            # 잔차 계산
            residual = y - y_pred_proba # 잔차 : 예측값과 실제값의 차이를 계산

            # 결정 트리 생성 및 학습
            tree = DecisionTreeRegressor(max_depth=self.max_depth) # 우리가 넣어준 데이터는 연속된 데이터이기 때문에 회귀모델로 사용
            tree.fit(X, residual) # 잔차를 회귀모델로 학습시킴 → 잔차를 줄여나가는 쪽으로 학습을 시키는 것이 가장 큰 핵심!
            self.trees.append(tree) # 트리 리스트에 추가
            
            # 트리 학습 결과를 반영한 예측값 업데이트 (점진적 개선)
            y_pred_log_odds += self.learning_rate * tree.predict(X)
            

    def predict(self, X):
        # 결국 우리가 예측을 결론 내리는 것은 0, 1
        return (self.predict_proba(X) >= 0.5).astype(int) # 확률값이 0.5 이상이면 1, 0.5 미만이면 0으로 예측

    def predict_proba(self, X):
        y_pred_log_odds = np.full((X.shape[0],), self.initial_log_odds)

        for tree in self.trees:
            y_pred_log_odds += self.learning_rate * tree.predict(X)

        return self.sigmoid(y_pred_log_odds)



In [15]:
# SimpleGradientBoostingClassifier로 유방암 데이터 예측

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 


# 유방암 데이터 로드 및 분리

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)

# 모델 생성 및 학습

simple_gb_clf = SimpleGradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3
)

simple_gb_clf.fit(X_train, y_train)

y_pred_train = simple_gb_clf.predict(X_train)
y_pred_test = simple_gb_clf.predict(X_test)

print(f"학습 정확도 : {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도 : {accuracy_score(y_test, y_pred_test)}")



학습 정확도 : 0.9929577464788732
평가 정확도 : 0.958041958041958


In [ ]:
### 와인데이터셋 이진 분류 <과제>

In [ ]:
# 1. 데이터로드
# 2. 데이터 분리
# 3. RandomForestClassifier 모델 생성 및 학습
# 4. 예측
# 5. 정확도 출력
# 6. 특성 중요도 시각화 .feature_importances_


In [ ]:
### 인간행동인식 다중 분류 <과제>

https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones 

사용자 행동 target class
- WALKING
- WALKING_UPSTAIRS
- WALKING_DOWNSTAIRS
- SITTING
- STANDING
- LAYING

**HAR 특성 설명**

| **특성 유형**             | **설명**                                                                                                                                     |
|--------------------------|--------------------------------------------------------------------------------------------------------------------------------------------|
| **Mean와 Std**            | 'tBodyAcc-mean()-X', 'tBodyAcc-std()-Y'와 같은 특성은 시간 도메인에서의 신체 가속도에 대한 평균 및 표준편차 값 ('X', 'Y', 'Z'는 축을 의미) |
| **주파수 도메인 특징**     | 이름에 'f'로 시작하는 특성은 주파수 도메인에서 계산된 값 (예를 들어 'fBodyAcc-meanFreq()-X'는 신체 가속도 센서의 X축에서의 평균 주파수 값)    |
| **가속도와 자이로스코프 데이터** | 'tBodyAcc', 'tGravityAcc', 'tBodyGyro' 등의 특성은 신체 가속도, 중력 가속도, 자이로스코프 데이터                                                  |
| **Jerk 신호**             | 'tBodyAccJerk-mean()-X'와 같은 특성은 가속도의 변화율 (활동 간의 차이를 분류하는 데 중요한 역할)                                          |
| **Magnitude(크기)**       | 'tBodyAccMag-mean()', 'tGravityAccMag-std()'와 같은 특성은 특정 축 방향의 가속도 또는 자이로스코프 값을 합친 크기                                             |
| **FFT 변환 기반 특징**    | 주파수 도메인에서의 데이터 특성들은 Fourier 변환을 통해 얻어짐 (예를 들어, 'fBodyAccMag-mean()'은 주파수 도메인에서 가속도의 크기의 평균값)                  |
| **Angle(각도)**           | 'angle(X,gravityMean)'과 같은 특성은 특정 축과 중력 벡터 간의 각도 (중력과의 상대적인 위치 나타냄)                                                    |
| **레이블**                | 'Activity' 컬럼에는 각 행의 활동 레이블이 포함 (걷기, 계단 오르기, 계단 내리기, 앉기, 서기, 눕기 등의 활동 종류 있음)                                          |


In [ ]:
# 1. 데이터 로드
# 2. 데이터 분리
# 3. 레이블 인코딩 (Activity 숫자로 변환)
# 4. 데이터 정규화
# 5. RandomForestClassifier 모델 생성 및 학습
# 6. 모델 예측
# 7. 평가(정확도)
# 8. 중요도 상위 20개의 특성 시각화


### GradientBoosting

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_breast_cancer()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

gb_clf = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=3
    )

gb_clf.fit(X_train, y_train)

y_pred_train = gb_clf.predict(X_train)
y_pred_test = gb_clf.predict(X_test)

print(f"학습 정확도 : {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도 : {accuracy_score(y_test, y_pred_test)}")

학습 정확도 : 0.9846153846153847
평가 정확도 : 0.9649122807017544


In [36]:
data.data.shape

(569, 30)

### HistGradientBoosting

- 고성능 GradientBoosting 모델로 대규모 데이터셋 처리에 적합
- 대규모 데이터셋에서는 메모리 사용량이 적고 학습 속도가 빠름
- Hiestogram 기반으로 256개의 구간으로 나누어 처리 병합하는 방식
- 결측치가 있어도 전처리가 필요 없음 → 모델 내부에서 하나의 범주를 설정해서 파악한다.
- LightGBM의 영향을 받아 만들어진 scikit-learn 모델

In [29]:
from sklearn.ensemble import HistGradientBoostingClassifier

hist_gb_clf = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=3,
    max_bins=255, # 255개의 구간으로 나누어 처리 병합하는 방식 → 모델 내부에서 하나의 범주를 설정해서 파악한다.
    # 255 → 1 개는 결측치 전용으로 사용
    early_stopping=True,    # 조기 중단 사용 여부
    n_iter_no_change=5      # 5번 이상 개선이 없으면 중단
    # 조기 중단 : 모델이 개선되지 않으면 중단
    # 반복 중 '일정 횟수' 이상 성능 향상이 없으면 학습 종료
    # '일정 횟수' 지정 (n_iter_no_change 기본값 10)
)

hist_gb_clf.fit(X_train, y_train)

y_pred_train = hist_gb_clf.predict(X_train)
y_pred_test = hist_gb_clf.predict(X_test)

print(f'학습 정확도: {accuracy_score(y_train, y_pred_train)}')
print(f'평가 정확도: {accuracy_score(y_test, y_pred_test)}')

학습 정확도: 0.9846153846153847
평가 정확도: 0.9649122807017544


In [ ]:
from sklearn.inspection import permutation_importance  # Permutation importance for feature evaluation
# 특성 중요도(=permutation importance) 시각화

# → DecisionTree 모델은 결국 '학습 데이터 → 내부 데이터'를 기반으로 활용해서 학습의 방향성을 잡아가는데 Classifier는 gini 지수 감소를 , Regeression는는 MSE 감소를 기준으로 학습을 진행한다.

# → permutation importance는 '외부 검증' 방식으로 특성 중요도를 측정한다.

result = permutation_importance(
    hist_gb_clf,
    X_train,
    y_train,
    n_repeats=5,
    random_state=0
)

# result : 왜 30개의 값이 나올까?
# → 30개의 특성이 있기 때문에 30개의 값이 나온다.
# data.data.shape : (569, 30) → 569개의 데이터가 30개의 특성을 가지고 있다.

result.importances # → 30 개의 '특성 중요도 값'을 보여준다
result.importances_std # → 30 개의 값의 '중요도 표준편차'를 보여준다
result.importances_mean # → 30 개의 값의 '중요도 평균'을 보여준다

# → feature 하나를 선택 → 값(row)를 무작위로 섞음 → 순서가 바뀌게됨 
# → 모델에 학습할 수 있게 제공 → 예측 정확도 하락 → 예측 정확도 하락 정도 측정 → 특성 중요도 측정

array([ 0.        ,  0.00175824,  0.        ,  0.        ,  0.        ,
        0.        , -0.00131868,  0.0043956 , -0.00131868,  0.        ,
        0.        ,  0.        ,  0.00131868,  0.00043956,  0.0021978 ,
        0.        ,  0.        , -0.00087912,  0.00087912,  0.        ,
       -0.00263736,  0.00175824, -0.00087912,  0.00835165, -0.0021978 ,
        0.        , -0.00131868,  0.02461538, -0.00131868,  0.        ])

In [ ]:
import matplotlib.pyplot as plt
# 특성 중요도 시각화

plt.figure(figsize=(10, 6))
plt.barh(data.feature_names, result.importances_mean)
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance')

In [37]:
### 회귀 모델

`sklearn.datasets.load_diabetes` 데이터셋
- 당뇨병 환자의 진단 자료를 바탕으로 만들어진 회귀용 데이터셋

**데이터셋 설명:**
- **목적**: 당뇨병 진행 정도(1년 후)를 예측
- **데이터 수**: 442개의 샘플
- **특성 수**: 10개의 특성 (10개의 입력 변수)
- **타겟**: 연속형 값, 당뇨병의 1년 후 진행 상황을 나타냄

**특성 설명:**
데이터셋의 각 특성(피처)은 환자의 다양한 생체 정보

1. **age**: 나이 (Age)
2. **sex**: 성별 (Sex)
3. **bmi**: 체질량 지수 (Body Mass Index)
4. **bp**: 평균 혈압 (Average Blood Pressure)
5. **s1**: 혈청 내 TC (Total Cholesterol)
6. **s2**: 혈청 내 LDL (Low-Density Lipoproteins)
7. **s3**: 혈청 내 HDL (High-Density Lipoproteins)
8. **s4**: 혈청 내 TCH (Total Cholesterol / HDL)
9. **s5**: 혈청 내 LTG (Log of Serum Triglycerides)
10. **s6**: 혈당 수치 (Blood Sugar Level)

**데이터 구조:**
- **입력 데이터 (data)**: (442, 10) 크기의 NumPy 배열
- **타겟 데이터 (target)**: (442,) 크기의 NumPy 배열로, 각 샘플의 타겟 값(연속형 값)을 포함

In [39]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [98]:
# 1. 데이터 로드
diabete = load_diabetes()
diabete
# 2. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(diabete.data, diabete.target, test_size=0.2, random_state=176466208)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 3. HistGradientBoostingRegressor 모델 생성 및 학습
hist_gb_reg = HistGradientBoostingRegressor(
    learning_rate=0.1,
    max_depth=3,
    max_bins=255,             # 255개의 구간으로 나누어 처리 병합하는 방식 → 모델 내부에서 하나의 범주를 설정해서 파악.
    max_iter=100,             # 최대 반복 횟수 설정
    random_state=0,           # 모델 내부에서 랜덤 시드를 설정해서 파악.
    l2_regularization=0.5,    # 정규화 강도 조절 → 과대적합 되지 않도록 조절.
    min_samples_leaf=5,       # 리프 노드의 최소 샘플 수 → 모델 내부에서 최소 샘플 수를 설정해서 파악.
    #DecisionTree 모델에서는 최소 샘플 수를 설정해서 파악.
    # early_stopping=True,
    # n_iter_no_change=5
)   # 비지도 학습때 평가, 검증 → 교차 검증 방식을 배우면 이해가 될 것이다.
# 하이퍼파라미터를 메뉴얼하게 설정해서 파악. → 여러개의 변수들이 있기 때문에 파악하기 쉽지 않다.

# 4. 학습 > 예측 > 평가 MSE, r2_score 계산
hist_gb_reg.fit(X_train, y_train)

y_train_pred = hist_gb_reg.predict(X_train)
y_test_pred = hist_gb_reg.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"학습 데이터 MSE: {mse_train}, R2: {r2_train}")
print(f"평가 데이터 MSE: {mse_test}, R2: {r2_test}")

학습 데이터 MSE: 1081.3074765780552, R2: 0.8110785121384774
평가 데이터 MSE: 2947.1310066790084, R2: 0.5627807449342701


In [ ]:
# 교차 검증

from sklearn.model_selection import GridSearchCV
# Cross Validation 방식으로 하이퍼파라미터 튜닝

param_grid = {
    'max_iter': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 1],
    'min_samples_leaf': [5, 10, 20],
    'l2_regularization': [0.01, 0.1, 1],
    'max_bins': [128, 191, 255]
}

hist_gb_reg = HistGradientBoostingRegressor(random_state=0)

grid_search = GridSearchCV(
    hist_gb_reg,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error'
    # n_jobs=-1,
    # verbose=2
)

grid_search.fit(X_train, y_train)

# param_grid로 전달해준 하이퍼 파라미터 중, 최고의 성능을 내는 하이퍼파라미터 조합합
grid_search.best_params_

{'l2_regularization': 0.1,
 'learning_rate': 0.01,
 'max_bins': 128,
 'max_depth': 3,
 'max_iter': 300,
 'min_samples_leaf': 20}

In [ ]:
# 최고의 성능을 내는 하이퍼 파라미터 조합으로 학습된 모델
best_hist_gb_reg = grid_search.best_estimator_
# best_hist_gb_reg # 최적의 하이퍼파라미터를 가진 모델

y_train_pred = best_hist_gb_reg.predict(X_train)
y_test_pred = best_hist_gb_reg.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"학습 데이터 MSE: {mse_train}, R2: {r2_train}")
print(f"평가 데이터 MSE: {mse_test}, R2: {r2_test}")

학습 데이터 MSE: 2172.2821222878224, R2: 0.6204680172041914
평가 데이터 MSE: 3108.7573350507973, R2: 0.538802800713408
